### Dependencies ###

In [33]:
import os
import torch
import numpy as np
import time
import pandas as pd

from scipy import spatial
print(os.getcwd())
print(os.listdir('../dataset/train'))

C:\projects\Lexical-Complexity-Prediction\baseline
['lcp_multi_train.tsv', 'lcp_single_train.tsv']


### Train data ###

In [34]:
train_single_tsv = '../dataset/train/lcp_single_train.tsv'
df_train_single = pd.read_csv(train_single_tsv, sep='\t', header=0)

In [35]:
print("Data columns: \n")
print(df_train_single.columns)
print("Total corpus len: {}".format(len(df_train_single)))
print("Subcorpus len:\n")
print(df_train_single['corpus'].value_counts())

Data columns: 

Index(['id', 'corpus', 'sentence', 'token', 'complexity'], dtype='object')
Total corpus len: 7662
Subcorpus len:

biomed      2576
bible       2574
europarl    2512
Name: corpus, dtype: int64


### Test data 

In [36]:
test_single_tsv = '../dataset/test/lcp_single_test.tsv'
df_test_single = pd.read_csv(test_single_tsv, sep='\t', header=0)


In [37]:
print("Data columns: \n")
print(df_test_single.columns)
print("Total corpus len: {}".format(len(df_test_single)))
print("Subcorpus len:\n")
print(df_test_single['corpus'].value_counts())
print(os.getcwd())

Data columns: 

Index(['id', 'corpus', 'sentence', 'token', 'complexity'], dtype='object')
Total corpus len: 917
Subcorpus len:

europarl    345
biomed      289
bible       283
Name: corpus, dtype: int64
C:\projects\Lexical-Complexity-Prediction\baseline


### Trial data 


In [38]:
trial_single_tsv = '../dataset/trial/lcp_single_trial.tsv'
df_trial_single = pd.read_csv(trial_single_tsv, sep='\t', header=0)


In [39]:
print("Data columns: \n")
print(df_trial_single.columns)
print("Total corpus len: {}".format(len(df_trial_single)))
print("Subcorpus len:\n")
print(df_trial_single['subcorpus'].value_counts())
print(os.getcwd())

Data columns: 

Index(['id', 'subcorpus', 'sentence', 'token', 'complexity'], dtype='object')
Total corpus len: 421
Subcorpus len:

europarl    143
bible       143
biomed      135
Name: subcorpus, dtype: int64
C:\projects\Lexical-Complexity-Prediction\baseline


### S-Bert

In [40]:
from sentence_transformers import SentenceTransformer

In [41]:
sbert_model = SentenceTransformer('stsb-distilbert-base')

In [42]:
def get_embedding_for_context(ctx):
    if not isinstance(ctx, list):
#       print("ctx is not list")
        ctx = [ctx]
    return sbert_model.encode(ctx)

start = time.time()
get_embedding_for_context("This is a test sentence")
print("Time for single prediction: {}".format(time.time() - start))

get_embedding_for_context(["This is a test sentence"] * 3000)
print("Time for pred of 3000 cases: {}".format(time.time() - start))

Time for single prediction: 0.41704225540161133
Time for pred of 3000 cases: 2.1432719230651855


### Handcrafted features

* Word length
* Syllable count
* Word frequency

In [53]:
import syllables
# According to the paper there are 3 handcrafted features
# - word length
# - word frequency 
# - syllable count
import csv
import math
import nltk

from collections import defaultdict
from nltk.stem.porter import PorterStemmer

reader = csv.reader(open('SUBTLEX.csv', 'r'))
frequency = defaultdict(float)
frequency_count = dict()
stemmer = PorterStemmer()

for row in reader:
    token = stemmer.stem(row[0].lower())
    frequency[token] += float(row[5])
    
#     if token in frequency_count:
#         frequency_count[token] += 1
#     else:
#         frequency_count[token] = 1
    
# for key in frequency:
#     frequency[key] = frequency[key]/frequency_count[key]

frequency = {k: math.log2(v) for k, v in frequency.items()}

def get_handcrafted_features(word):
    word = str(word)
    return [len(word), syllables.estimate(word), frequency.get(stemmer.stem(word.lower())) or 0]

get_handcrafted_features("Basketball")


[10, 3, 4.4409521980296365]

### Load datasets

In [54]:
from torch.utils.data import Dataset

def preprocess_embeddings(dataset):
    # Preprocess all sentence embeddings for the data:
    sentence_embeddings = {}
    
    all_sentences = dataset['sentence'].tolist()

    start = time.time()
    all_sentence_embeddings = get_embedding_for_context(all_sentences)
    print("Encoding time for all sentences: {}".format(time.time() - start))
    return all_sentence_embeddings
    

class CompLexDataset(Dataset):
    global dataset_type
    
    def __init__(self, dataset_type):
        self.dataset_type = dataset_type
        
        if(self.dataset_type == 'train'):                   
            self.all_sentence_embeddings = preprocess_embeddings(df_train_single)
        elif(self.dataset_type == 'trial'):
            self.all_sentence_embeddings = preprocess_embeddings(df_trial_single)
        elif(self.dataset_type == 'test'):
            self.all_sentence_embeddings = preprocess_embeddings(df_test_single)
    
    def __len__(self):
        if(self.dataset_type == 'train'):                   
            return len(df_train_single)
        elif(self.dataset_type == 'trial'):
            return len(df_trial_single)
        elif(self.dataset_type == 'test'):
            return len(df_test_single)
        else: 
            raise Exception("Invalid dataset type.", self.dataset_type)

    
    def __getitem__(self,idx):
        start = time.time()
        if(self.dataset_type == 'train'):
            token = df_train_single.loc[idx, 'token']
            token = str(token)
            out = df_train_single.loc[idx, 'complexity']
        elif(self.dataset_type == 'trial'):
            token = df_trial_single.loc[idx, 'token']
            token = str(token)
            out = df_trial_single.loc[idx, 'complexity']
        elif(self.dataset_type == 'test'):
            token = df_test_single.loc[idx, 'token']
            token = str(token)
            out = df_test_single.loc[idx, 'complexity']
        else: 
            raise Exception("Invalid dataset type.", self.dataset_type)
        
        handcrafted_features = get_handcrafted_features(token)

        sentence_ctx = self.all_sentence_embeddings[idx]
        
#         if token.lower() in glove_w2v:   
#             w2v_for_token = glove_w2v[token.lower()]
#         else:
#             #print("Token {} not found".format(token.lower()))
#             w2v_for_token = [0] * 300
        
        
        result = {
            'inp': torch.from_numpy(np.hstack((np.array(handcrafted_features), sentence_ctx)).ravel()).float(), 
            'out': torch.from_numpy(np.array([out])).float()
        }
        
        #print("Idx {} fetch time: {}".format(idx, time.time() - start))
        return result
    

In [55]:
train_dataset = CompLexDataset("train")
trial_dataset = CompLexDataset("trial")
test_dataset = CompLexDataset("test")

Encoding time for all sentences: 25.441973209381104
Encoding time for all sentences: 1.4798219203948975
Encoding time for all sentences: 3.0330488681793213


### Network ###

In [56]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        
        self.fc1 = nn.Linear(len(train_dataset[0]['inp']), 1600)
        self.fc2 = nn.Linear(1600, 1600)
        self.fc3 = nn.Linear(1600, 1)

        self.sigmoid = nn.Sigmoid() 


    def forward(self,x):

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        #return x
        return self.sigmoid(x)
        
    
net = Network()

In [57]:
def train(model, x, y, optimizer, criterion):
    model.zero_grad()
    output = model(x)
 #   print("output", output)
 #   print("y", y)
    loss = criterion(output,y)
    loss.backward()
    optimizer.step()

    return loss, output


In [58]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())

True
0


### Mean Squared Error ###
Training phase

In [59]:
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import time

criterion = nn.MSELoss()
EPOCHS = 30
BATCH_SIZE = 64
optm = Adam(net.parameters(), lr = 0.00001)

data_train = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(EPOCHS):
    epoch_loss = 0
    correct = 0
    
    for bidx, batch in enumerate(data_train):
        #start = time.time()
        x_train = batch['inp']
        y_train = batch['out']
        #print("Fetch time: {}".format(time.time() - start))
        
        #start = time.time()
        loss, predictions = train(net,x_train,y_train, optm, criterion)
        epoch_loss += loss
        #print("Predict time: {}".format(time.time() - start))
        
    print('Epoch {} Loss : {}'.format((epoch+1),epoch_loss))

Epoch 1 Loss : 1.5881422758102417
Epoch 2 Loss : 1.1757149696350098
Epoch 3 Loss : 1.153933048248291
Epoch 4 Loss : 1.1381193399429321
Epoch 5 Loss : 1.1303942203521729
Epoch 6 Loss : 1.1176087856292725
Epoch 7 Loss : 1.1048948764801025
Epoch 8 Loss : 1.1002025604248047
Epoch 9 Loss : 1.0955835580825806
Epoch 10 Loss : 1.0947943925857544
Epoch 11 Loss : 1.0883023738861084
Epoch 12 Loss : 1.0804123878479004
Epoch 13 Loss : 1.080855131149292
Epoch 14 Loss : 1.073858618736267
Epoch 15 Loss : 1.069102168083191
Epoch 16 Loss : 1.0711023807525635
Epoch 17 Loss : 1.0651047229766846
Epoch 18 Loss : 1.0682086944580078
Epoch 19 Loss : 1.064536690711975
Epoch 20 Loss : 1.058791995048523
Epoch 21 Loss : 1.0599945783615112
Epoch 22 Loss : 1.0582911968231201
Epoch 23 Loss : 1.0593011379241943
Epoch 24 Loss : 1.0588645935058594
Epoch 25 Loss : 1.0510766506195068
Epoch 26 Loss : 1.0501083135604858
Epoch 27 Loss : 1.0516985654830933
Epoch 28 Loss : 1.055222511291504
Epoch 29 Loss : 1.052940845489502
Ep

### Output for single sample

In [60]:
net(train_dataset[210]['inp'])

tensor([0.2263], grad_fn=<SigmoidBackward>)

### Mean Absolute Error ###

#### MAE for test dataset

In [ ]:
# from sklearn.metrics import mean_absolute_error

# y_true = [test_dataset[i]['out'].item() for i in range(len(test_dataset))]
# y_pred = []

# test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = False)
# for bidx, batch in enumerate(test_loader):
#         #start = time.time()
#         x_train = batch['inp']
#         y_pred.append(net(x_train))

# y_pred = [x.item() for i in range(len(y_pred)) for x in y_pred[i] ]

# mae = mean_absolute_error(y_true, y_pred)
# print("MAE for test data: ", mae)

# with open('test_results.csv', 'w', newline='') as f:
#     f_writer = csv.writer(f, delimiter=',',)
#     for idx in range(len(df_test_single)):
#        f_writer.writerow((df_test_single.loc[idx, 'id'], str(y_pred[idx])))


#### MAE for trial dataset

In [61]:
from sklearn.metrics import mean_absolute_error

y_true = [trial_dataset[i]['out'].item() for i in range(len(trial_dataset))]
y_pred = []

trial_loader = DataLoader(dataset = trial_dataset, batch_size = BATCH_SIZE, shuffle = False)
for bidx, batch in enumerate(trial_loader):
        #start = time.time()
        x_train = batch['inp']
        y_pred.append(net(x_train))

y_pred = [x.item() for i in range(len(y_pred)) for x in y_pred[i] ]

mae = mean_absolute_error(y_true, y_pred)
print("MAE for trial data: ", mae)

MAE for trial data:  0.07371862768029656


#### MAE for train dataset

In [62]:
from sklearn.metrics import mean_absolute_error

y_true = [train_dataset[i]['out'].item() for i in range(len(train_dataset))]
y_pred = []

test_loader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = False)
for bidx, batch in enumerate(test_loader):
        #start = time.time()
        x_train = batch['inp']
        y_pred.append(net(x_train))

y_pred = [x.item() for i in range(len(y_pred)) for x in y_pred[i] ]

mae = mean_absolute_error(y_true, y_pred)
print("MAE for train data: ", mae)

MAE for train data:  0.07115381368572299


#### MAE for total random

In [63]:
from sklearn.metrics import mean_absolute_error
import random

y_true = [train_dataset[i]['out'].item() for i in range(len(train_dataset))]
y_pred = [random.random() for i in range(len(train_dataset))]


mae = mean_absolute_error(y_true, y_pred)
print("Mean Absolute Error for train data: ", mae)

Mean Absolute Error for train data:  0.30674491430737033


#### Demo

In [65]:
def prepare_sentence(sentence, token):
    sentence_embeddings = get_embedding_for_context(sentence)[0]
    handcrafted_features = get_handcrafted_features(token)
            

    return {
            'inp': torch.from_numpy(np.hstack((np.array(handcrafted_features), sentence_embeddings))).ravel().float() 
           }

    

In [66]:
sentence = 'Peter loves pineapples and apples!'
token1 = 'pineapples'
token2 = 'apples'

y_pred1 = net(prepare_sentence(sentence, token1)['inp'])
print('SENTENCE: ', sentence, '\nTOKEN: ', token1, '\nCOMPLEXITY: ', y_pred1.item(), '\n')

y_pred2 = net(prepare_sentence(sentence, token2)['inp'])
print('SENTENCE: ', sentence, '\nTOKEN: ', token2, '\nCOMPLEXITY: ', y_pred2.item(), '\n')



SENTENCE:  Peter loves pineapples and apples! 
TOKEN:  pineapples 
COMPLEXITY:  0.34796950221061707 

SENTENCE:  Peter loves pineapples and apples! 
TOKEN:  apples 
COMPLEXITY:  0.2773303985595703 

